In [1]:
import os
os.chdir('/Users/aleksandrberezutskii/Desktop/GitHub/MPOpt/')
from mpopt.contractor.contractor import apply_one_site_unitary, apply_two_site_unitary, mps_mpo_contract
from mpopt.mps.explicit import *
from mpopt.mps.canonical import *
from mpopt.utils.utils import *
from tests.mps.test_explicit import _create_psi
from experiments.decoding_classical import *
import numpy as np
import qecstruct as qec

# TODO's
# check out the sanity tests they have in TenPy
# check out typing for fixing types of variables in functions
# dtypes: complex64/32? float64/32?
# bond dimensions check: mpss and mpos
# compress method for mps

In [2]:
XOR_BULK = np.zeros((2, 2, 2, 2))
for i in range(2):
    for j in range(2):
        for k in range(2):
            for l in range(2):
                XOR_BULK[i, j, k, l] = (i ^ j ^ k ^ 1) * np.eye(2)[k, l]

XOR_LEFT = np.zeros((1, 2, 2, 2))
for j in range(2):
    for k in range(2):
        for l in range(2):
            XOR_LEFT[0, j, k, l] = np.eye(2)[j, k] * np.eye(2)[k, l]

XOR_RIGHT = np.zeros((2, 1, 2, 2))
for i in range(2):
    for k in range(2):
        for l in range(2):
            XOR_RIGHT[i, 0, k, l] = np.eye(2)[i, k] * np.eye(2)[k, l]

SWAP = np.zeros((2, 2, 2, 2))
for i in range(2):
    for j in range(2):
        for k in range(2):
            for l in range(2):
                SWAP[i, j, k, l] = np.eye(2)[i, j] * np.eye(2)[k, l]

tensors = [XOR_LEFT, XOR_BULK, SWAP, XOR_RIGHT]

In [71]:
NUM_BITS = 4
NUM_CHECKS = 3
BIT_DEGREE = 3
CHECK_DEGREE = 4
PROB_CHANNEL = 0.1
PROB_ERROR = 0.1
if NUM_BITS / NUM_CHECKS != CHECK_DEGREE / BIT_DEGREE:
    raise ValueError("The graph must be bipartite.")

# Constructing the code as a qecstruct object.
example_code = qec.random_regular_code(
    NUM_BITS, NUM_CHECKS, BIT_DEGREE, CHECK_DEGREE, qec.Rng()
)

# Getting the sites for which the constraints should be applied.
sites_all = linear_code_checks(example_code)

# Rebuilding the sites lists as used in the ConstrainedString class.
strings = []
for _, sublist in enumerate(sites_all):

    # Retreiving the sites indices where we apply the "bulk"/"boundary" XOR tensors.
    xor_left_sites = [sublist[0]]
    xor_bulk_sites = [sublist[i] for i in range(1, CHECK_DEGREE - 1)]
    xor_right_sites = [sublist[-1]]

    # Retreiving the sites indices where we apply the SWAP tensors.
    swap_sites = list(range(sublist[0] + 1, sublist[-1]))
    for k in range(1, CHECK_DEGREE - 1):
        swap_sites.remove(sublist[k])

    strings.append([xor_left_sites, xor_bulk_sites, swap_sites, xor_right_sites])

# Building an initial and a perturbed codeword.
INITIAL_CODEWORD = example_code.random_codeword(qec.Rng())
PERTURBED_CODEWORD = INITIAL_CODEWORD + qec.BinarySymmetricChannel(
    PROB_ERROR
).sample(NUM_BITS, qec.Rng())
INITIAL_CODEWORD = "".join(str(bit) for bit in bin_vec_to_dense(INITIAL_CODEWORD))
PERTURBED_CODEWORD = "".join(
    str(bit) for bit in bin_vec_to_dense(PERTURBED_CODEWORD)
)

# Building the corresponding MPSs.
initial_codeword_state = create_custom_product_state(
    INITIAL_CODEWORD
).to_right_canonical()
perturbed_codeword_state = create_custom_product_state(
    PERTURBED_CODEWORD
).to_right_canonical()

# Passing the perturbed codeword state through the bias channel.
bias_channel = [binary_symmetric_channel(PROB_CHANNEL) for _ in range(NUM_BITS)]
perturbed_codeword_state = mps_mpo_contract(
    perturbed_codeword_state, bias_channel, 0
)

# Passing the perturbed codeword state through the parity constraints defined by the code.
for i in range(NUM_CHECKS):

    orth_centre_init = find_orth_centre(perturbed_codeword_state)[0]

    constraint_string = ConstraintString(tensors, strings[i])
    constraint_mpo = constraint_string.get_mpo()

    START_SITE = min(constraint_string.flat())

    perturbed_codeword_state = move_orth_centre(
        perturbed_codeword_state, orth_centre_init, START_SITE
    )
    perturbed_codeword = mps_mpo_contract(
        perturbed_codeword_state, constraint_mpo, START_SITE
    )

# Building the density matrix MPO.
density_mpo = to_density_mpo(perturbed_codeword_state)

print("DMRG running:")

# Creating a random product state to start the DMRG with.
INIT_STATE_DMRG = "".join(
    str(bit) for bit in np.random.randint(low=0, high=2, size=NUM_BITS, dtype=np.int32)
)
mps_dmrg_start = create_custom_product_state(INIT_STATE_DMRG)

engine = dmrg(mps_dmrg_start, density_mpo, chi_max=128, cut=1e-14, mode="LA")
engine.run(20)
mps_dmrg_final = engine.mps.to_right_canonical()

print(
    "The overlap of the density MPO main component and the initial codeword state: ",
    inner_product(mps_dmrg_final, initial_codeword_state),
)
print(
    "__________________________________________________________________________________________"
)

DMRG running:


100%|██████████| 20/20 [00:00<00:00, 36.04it/s]

The overlap of the density MPO main component and the initial codeword state:  0.8100000000000002
__________________________________________________________________________________________


In [4]:
from more_itertools import powerset

In [65]:
def get_words(code):
    """
    Return the list of codewords of a linear code. Codewords are returned
    as integers in most-significant-bit-first convention.

    Arguments:
        code : qecstruct.LinearCode
            Linear code object.

    Returns:
        codewords : list of ints
            List of codewords.
    """

    length = code.length()
    codewords = []

    for word in range(2 ** length):
        msg = np.array(list(np.binary_repr(word, length)), dtype=np.int32)
        vec = qec.BinaryVector(length, np.flatnonzero(msg))
        if code.has_codeword(vec):
            codewords.append(word)

    return np.array(codewords)

In [48]:
codewords = []
gen_mat = example_code.gen_mat()
rows_bin = gen_mat.rows()
rows_dense = [bin_vec_to_dense(row_bin) for row_bin in rows_bin]
rows_int = [row.dot(1 << np.arange(row.size)[::-1]) for row in rows_dense]

for gens in powerset(rows_int):
    codewords.append(sum(gens))

In [43]:
qec.BinaryVector(NUM_BITS, np.flatnonzero(rows_dense[0]))

[0, 1]

In [45]:
gen_mat

[0, 1]
[0, 2]
[0, 3]

In [98]:
[np.binary_repr(x, NUM_BITS) for x in codewords]

/var/folders/l8/sv77sl0155l_p56w23fb7vlc0000gn/T/ipykernel_81122/3142374927.py:1: DeprecationWarning: Insufficient bit width provided. This behavior will raise an error in the future.
  [np.binary_repr(x, NUM_BITS) for x in codewords]


['0000', '1100', '1010', '1001', '10110', '10101', '10011', '11111']

In [88]:
[np.binary_repr(x, NUM_BITS) for x in get_words(example_code)]

['0000', '0011', '0101', '0110', '1001', '1010', '1100', '1111']

In [97]:
for i in range(100000):
    cand = ''.join(str(x) for x in bin_vec_to_dense(example_code.random_codeword(qec.Rng())))
    if cand not in [np.binary_repr(x, NUM_BITS) for x in get_words(example_code)]:
        print(False, cand)

In [ ]:
[np.binary_repr(x, width=NUM_BITS) for x in get_codewords(example_code)]